In [1]:
import sys, os
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [3]:
df=pd.read_csv('../input/fer2013/fer2013.csv')


In [4]:
X_train,train_y,X_test,test_y=[],[],[],[]


In [5]:
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [6]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 30
width, height = 48, 48


In [7]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [8]:
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)


In [9]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)


In [10]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)


In [11]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))


In [12]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [13]:
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/30
449/449 [==============================] - 244s 542ms/step - loss: 1.7330 - accuracy: 0.2843 - val_loss: 1.6159 - val_accuracy: 0.3516
Epoch 2/30
449/449 [==============================] - 243s 542ms/step - loss: 1.5168 - accuracy: 0.4046 - val_loss: 1.4162 - val_accuracy: 0.4452
Epoch 3/30
449/449 [==============================] - 241s 537ms/step - loss: 1.4083 - accuracy: 0.4517 - val_loss: 1.3661 - val_accuracy: 0.4831
Epoch 4/30
449/449 [==============================] - 242s 540ms/step - loss: 1.3475 - accuracy: 0.4787 - val_loss: 1.3337 - val_accuracy: 0.4823
Epoch 5/30
449/449 [==============================] - 241s 537ms/step - loss: 1.3051 - accuracy: 0.4956 - val_loss: 1.2684 - val_accuracy: 0.5177
Epoch 6/30
449/449 [==============================] - 240s 535ms/step - loss: 1.2651 - accuracy: 0.5119 - val_loss: 1.2515 - val_accuracy: 0.5227
Epoch 7/30
449/449 [==============================] - 240s 535ms/step - loss: 1.2390 - accuracy: 0.5268 - val_loss: 1.2124 -

In [14]:
#Saving the  model to  use it later on
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")
